In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Put the archived tablature dataset in the following drive location: /content/drive/MyDrive/cs1430/chord-classifier/tablature_dataset.7z

In [2]:
%cd /content
!cp /content/drive/MyDrive/cs1430/chord-classifier/tablature_dataset.7z /content/tablature_dataset.7z
!7z x tablature_dataset.7z


/content

7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,8 CPUs Intel(R) Xeon(R) CPU @ 2.00GHz (50653),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 3678389168 bytes (3508 MiB)

Extracting archive: tablature_dataset.7z
--
Path = tablature_dataset.7z
Type = 7z
Physical Size = 3678389168
Headers Size = 16228
Method = Delta LZMA2:24
Solid = +
Blocks = 2

  0%      0% 5        0% 7 - tablature_dataset/tablature_frames/0_10.png                                                      0% 9 - tablature_dataset/tablature_frames/0_101.png                                                       0% 11 - tablature_dataset/tablature_frames/0_103.

Compute std and mean of the dataset for standardization.

Load dataset into memory.

In [4]:
import os
from PIL import Image
import numpy as np
from tqdm import tqdm

image_folder = 'tablature_dataset/tablature_frames'
image_files = [f for f in os.listdir(image_folder) if f.endswith('.png')]
image_files.sort(key=lambda x: (int(x.split('_')[0]), int(x.split('_')[1].split('.')[0])))
image_files = np.stack(image_files)

# Load all images into memory for fast access.
global_image_arr = []

for i, image_file in enumerate(tqdm(image_files)):
  img_path = os.path.join(image_folder, image_file)
  image = Image.open(img_path).convert('RGB')
  image = Image.open(img_path).convert('L')
  image = image.crop((image.size[0]/3, image.size[1]/2, image.size[0], image.size[1]))
  image = image.resize((320, 270))
  image = np.array(image)
  image = image / 255.0
  global_image_arr.append(image)

global_image_arr = np.stack(global_image_arr)

100%|██████████| 1995/1995 [05:28<00:00,  6.07it/s]


In [5]:
mean = np.mean(global_image_arr, axis=0)
std = np.std(global_image_arr, axis=0)

np.save('mean_std.npy', (mean, std))

In [6]:
import os
import numpy as np
from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from sklearn.model_selection import train_test_split

class GuitarTabDataset(Dataset):
    def __init__(self, image_folder, label_folder, indices, transform=None):
        self.image_folder = image_folder
        self.label_folder = label_folder
        self.transform = transform
        if self.transform == None:
          self.transform = transforms.Compose([
              transforms.ToTensor(),
              transforms.RandomResizedCrop(size=(270, 320), scale=(0.9, 1.0), ratio=(0.9, 1.2)),
              transforms.ColorJitter(brightness=0.1, contrast=0.1),
              transforms.Normalize(mean, std)
          ])

        # Image files are loaded and sorted above in global_image_arr

        # Load all labels
        self.labels = []
        for i in range(25):  # Assuming 24 npy files corresponding to batches
            self.labels.extend(np.load(os.path.join(label_folder, f"{i}.npy")))
        self.labels = np.stack(self.labels).astype(np.float32)

        # Filter files and labels based on provided indices
        self.image_files = global_image_arr[indices]
        self.labels = self.labels[indices]

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        image = self.image_files[idx]
        if self.transform:
          image = self.transform(image)
        image = image.type(torch.float32)
        label = self.labels[idx]

        # Hack to make open
        open_strings = (np.sum(label, axis=1) == 0)
        label[open_strings, 0] = 1
        return image, label

In [7]:
from sklearn.model_selection import train_test_split

# Assuming all files are loaded to calculate indices
all_image_files = [f for f in os.listdir('tablature_dataset/tablature_frames') if f.endswith('.png')]
all_image_files.sort(key=lambda x: (int(x.split('_')[0]), int(x.split('_')[1].split('.')[0])))
all_indices = list(range(len(all_image_files)))

# Perform the split
train_indices, val_indices = train_test_split(all_indices, test_size=0.2, random_state=42)
transform = None

# Create dataset instances
train_dataset = GuitarTabDataset(image_folder='tablature_dataset/tablature_frames', label_folder='tablature_dataset/tablature_labels_converted', indices=train_indices, transform=transform)
val_dataset = GuitarTabDataset(image_folder='tablature_dataset/tablature_frames', label_folder='tablature_dataset/tablature_labels_converted', indices=val_indices, transform=transform)

Inspect the dataset.

In [8]:
import torchvision

image, label = train_dataset[8]
# Unnormalize image
image = (image * std) + mean
image = torchvision.transforms.functional.to_pil_image(image)

#image
label


array([[0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.]], dtype=float32)

In [ ]:
import torch.optim as optim

class GuitarTabCNN(nn.Module):
    def __init__(self):
        super(GuitarTabCNN, self).__init__()
        self.sequential = nn.Sequential(
          nn.Conv2d(1, 64, kernel_size=11, stride=3),
          nn.BatchNorm2d(64),
          nn.ReLU(),
          nn.MaxPool2d(kernel_size=3, stride=2, padding=0),
          nn.Conv2d(64, 128, kernel_size=7, stride=1, padding=2),
          nn.BatchNorm2d(128),
          nn.ReLU(),
          nn.MaxPool2d(kernel_size=3, stride=2, padding=0),
          nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
          nn.BatchNorm2d(256),
          nn.ReLU(),
          nn.Conv2d(256, 384, kernel_size=3, stride=1, padding=1),
          nn.BatchNorm2d(384),
          nn.ReLU(),
          nn.MaxPool2d(kernel_size=3, stride=2, padding=0),
          nn.Conv2d(384, 384, kernel_size=3, stride=1, padding=1),
          nn.BatchNorm2d(384),
          nn.ReLU(),
          nn.Conv2d(384, 384, kernel_size=3, stride=1, padding=1),
          nn.BatchNorm2d(384),
          nn.ReLU(),
          nn.Conv2d(384, 256, kernel_size=3, stride=1, padding=1),
          nn.BatchNorm2d(256),
          nn.ReLU(),
          nn.MaxPool2d(kernel_size=3, stride=2, padding=0),
          nn.Flatten(),
          nn.Linear(5120, 2000),
          nn.BatchNorm1d(2000),
          nn.ReLU(),
          nn.Linear(2000, 1000),
          nn.BatchNorm1d(1000),
          nn.ReLU(),
          nn.Linear(1000, 1000),
          nn.BatchNorm1d(1000),
          nn.ReLU(),
          nn.Linear(1000, 6 * 21)
        )


    def forward(self, x):
        x = self.sequential(x)

        # Softmax each set of 21 fret logits. This represents the probability that a particular fret is pressed for that string.
        x = torch.nn.functional.softmax(x.view(-1, 21), dim=1).view(-1, 6*21)
        return x

model = GuitarTabCNN()

# Optimizer
optimizer = optim.Adam(model.parameters(), lr=2e-5)
# optimizer = optim.SGD(model.parameters(), lr=1e-4, momentum=0.9)

# Loss function
# Weight the loss higher on non-open strings, since most strings are open which leads to a degenerate solution.
weight_vec = torch.ones(21, dtype=torch.float32)
weight_vec[0] = 0.04
criterion = nn.CrossEntropyLoss(weight=weight_vec)

In [ ]:
!pip install torcheval

import torch.optim as optim
from tqdm import tqdm
import torcheval.metrics as metrics

# Dataloaders
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'Total trainable parameters: {total_params}')

acc = metrics.MulticlassAccuracy()
pressed_acc = metrics.MulticlassAccuracy()
open_acc = metrics.MulticlassAccuracy()

# Training loop with validation
for epoch in range(30):  # number of epochs
    print(f'Epoch {epoch+1}:')
    model.train()
    images_total = 0.0
    running_loss = 0.0
    progress_bar = tqdm(train_loader)
    for i, (images, labels) in enumerate(progress_bar):
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs.view(-1, 21), labels.view(-1, 21))
        loss.backward()
        optimizer.step()

        string_outputs = outputs.view(-1, 21)
        string_labels = labels.view(-1, 21)
        pressed_labels = (string_labels[:, 0] != 1)
        open_labels = (string_labels[:, 0] == 1.0)

        acc.update(torch.argmax(string_outputs, dim=1), torch.argmax(string_labels, dim=1))
        open_acc.update(torch.argmax(string_outputs[open_labels], dim=1), torch.argmax(string_labels[open_labels], dim=1))
        pressed_acc.update(torch.argmax(string_outputs[pressed_labels], dim=1), torch.argmax(string_labels[pressed_labels], dim=1))
        running_loss += loss.item() * images.shape[0]
        images_total += images.shape[0]
        progress_bar.set_description(f"Loss: {(running_loss / images_total):.4f}, Acc: {(acc.compute()):.4f}, Pressed Acc: {(pressed_acc.compute()):.4f}, Open Acc: {(open_acc.compute()):.4f}")

    acc.reset()
    pressed_acc.reset()
    open_acc.reset()

    # Validation
    model.eval()
    val_loss = 0.0
    val_images_total = 0.0
    with torch.no_grad():
        for images, labels in val_loader:
            outputs = model(images)
            loss = criterion(outputs.view(-1, 21), labels.view(-1, 21))
            val_loss += loss.item() * images.shape[0]
            val_images_total += images.shape[0]

            string_outputs = outputs.view(-1, 21)
            string_labels = labels.view(-1, 21)
            pressed_labels = (string_labels[:, 0] != 1)
            open_labels = (string_labels[:, 0] == 1)

            acc.update(torch.argmax(string_outputs, dim=1), torch.argmax(string_labels, dim=1))
            pressed_acc.update(torch.argmax(string_outputs[pressed_labels], dim=1), torch.argmax(string_labels[pressed_labels], dim=1))
            open_acc.update(torch.argmax(string_outputs[open_labels], dim=1), torch.argmax(string_labels[open_labels], dim=1))
    print(f"Epoch {epoch+1}, Training Loss: {(running_loss / images_total):.4f}, Validation Loss: {(val_loss / val_images_total):.4f}, Acc: {(acc.compute()):.4f}, Pressed Acc: {(pressed_acc.compute()):.4f}, Open Acc: {(open_acc.compute()):.4f}")
    acc.reset()
    pressed_acc.reset()
    open_acc.reset()

    torch.save(model.state_dict(), 'model.pt')

Total trainable parameters: 18511438
Epoch 1:


Loss: 0.6894, Acc: 0.8178, Pressed Acc: 0.9039, Open Acc: 0.7846: 100%|██████████| 200/200 [00:38<00:00,  5.25it/s]


Epoch 1, Training Loss: 0.6894, Validation Loss: 0.6773, Acc: 0.7853, Pressed Acc: 0.8643, Open Acc: 0.7564
Epoch 2:


Loss: 0.6888, Acc: 0.8190, Pressed Acc: 0.9043, Open Acc: 0.7862: 100%|██████████| 200/200 [00:39<00:00,  5.06it/s]


Epoch 2, Training Loss: 0.6888, Validation Loss: 0.6761, Acc: 0.7924, Pressed Acc: 0.8674, Open Acc: 0.7650
Epoch 3:


...

Epoch 28, Training Loss: 0.6799, Validation Loss: 0.6737, Acc: 0.8012, Pressed Acc: 0.8690, Open Acc: 0.7764
Epoch 29:


Loss: 0.6819, Acc: 0.8495, Pressed Acc: 0.9249, Open Acc: 0.8205: 100%|██████████| 200/200 [00:37<00:00,  5.28it/s]


Epoch 29, Training Loss: 0.6819, Validation Loss: 0.6676, Acc: 0.8145, Pressed Acc: 0.8939, Open Acc: 0.7855
Epoch 30:


Loss: 0.6791, Acc: 0.8517, Pressed Acc: 0.9358, Open Acc: 0.8193: 100%|██████████| 200/200 [00:37<00:00,  5.26it/s]


Epoch 30, Training Loss: 0.6791, Validation Loss: 0.6629, Acc: 0.8283, Pressed Acc: 0.9080, Open Acc: 0.7992
